In [3]:
%load_ext autoreload

In [4]:
%autoreload 2

import sys
sys.path.append('..')

from jax import numpy as jnp
import jax
import time
from scipy.integrate import quad, dblquad
import equinox as eqx
import jax_cosmo as jc
from quadax import quadgk, quadcc, quadts, romberg
from functools import partial

In [5]:
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import matplotlib.animation as animation 
from numpy import loadtxt
from diffrax import Kvaerno5, Tsit5
%matplotlib inline

from plot_params import params
#pylab.rcParams.update(params)

cols_default = plt.rcParams['axes.prop_cycle'].by_key()['color']

In [6]:
from linx.background import BackgroundModel
from linx.nuclear import NuclearRates 
from linx.abundances import AbundanceModel
from linx.reactions import Reaction
from linx.thermo import T_g 
from linx.const import mn, mp, eta0, hbar, me, aFS
from scipy.integrate import odeint, solve_ivp, simpson, cumulative_simpson
from linx.pd_rates import decay_model
from linx.injected_spec import InjectedSpectrum
eps = 1e-3
Emin = 1.5 # MeV
Ephb_T_max = 200

In [7]:
bkg_model = BackgroundModel()
t_vec, a_vec, rho_g_vec, rho_nu_vec, rho_NP_vec, p_NP_vec, Neff_vec = bkg_model(jnp.asarray(0.), T_end = 1e-4, max_steps = 512*4)

`\         /´  ||||        ||||  |||||     ||||  ||||   ||||
 /\_______/\   ||||        ||||  |||||||   ||||   |||| ||||
 ) __` ´__ (   ||||        ||||  |||| |||| ||||    |||||||
/  `-|_|-´  \  ||||        ||||  ||||  |||| |||    ||||||| 
/   (_x_)   \  ||||||||||  ||||  ||||   |||||||   |||| ||||
  )  `-´  (    ||||||||||  ||||  ||||    ||||||  ||||   ||||
 
Compiling thermodynamics model...


/home/mkwok/miniconda3/envs/linx/lib/python3.13/site-packages/equinox/_jit.py:55: UserWarning: `diffrax.diffeqsolve(..., discrete_terminating_event=...)` is deprecated in favour of the more general `diffrax.diffeqsolve(..., event=...)` interface. This will be removed in some future version of Diffrax.
  out = fun(*args, **kwargs)


In [8]:
InjSpec = InjectedSpectrum()
E_grid = jnp.linspace(1.5, jnp.exp(3), 100)
mod = decay_model(60, 1e5, 5e-7, 1, 0, t_vec, a_vec, rho_g_vec)

In [107]:
mod.temp()

AttributeError: 'decay_model' object has no attribute 'temp'

In [105]:
dir(mod)

['Be7gHe3He4_cross',
 'Be7gHe3He4_frwrd_rate',
 'Be7gpLi6_cross',
 'Be7gpLi6_frwrd_rate',
 'Be7gppnHe4_cross',
 'Be7gppnHe4_frwrd_rate',
 'E0',
 'He3gnpp_cross',
 'He3gnpp_frwrd_rate',
 'He3gpd_cross',
 'He3gpd_frwrd_rate',
 'He4gdd_cross',
 'He4gdd_frwrd_rate',
 'He4gnHe3_cross',
 'He4gnHe3_frwrd_rate',
 'He4gnpd_cross',
 'He4gnpd_frwrd_rate',
 'He4gpt_cross',
 'He4gpt_frwrd_rate',
 'Li6gX_cross',
 'Li6gX_frwrd_rate',
 'Li6gnpHe4_cross',
 'Li6gnpHe4_frwrd_rate',
 'Li7gnLi6_cross',
 'Li7gnLi6_frwrd_rate',
 'Li7gnnpHe4_cross',
 'Li7gnnpHe4_frwrd_rate',
 'Li7gtHe4_cross',
 'Li7gtHe4_frwrd_rate',
 'S_electron_0',
 'S_electron_cont',
 'S_photon_0',
 'S_photon_cont',
 'T_g_vec',
 '__abstractclassvars__',
 '__abstractmethods__',
 '__abstractvars__',
 '__annotations__',
 '__class__',
 '__dataclass_fields__',
 '__dataclass_params__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__firstlineno__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '_

In [131]:
InjSpec.get_spectrum(5, mod.get_source_0(), mod.get_source_cont(), 0.01)

Rate time:  21.40584945678711
Kernel time:  0.43370509147644043
Source time:  0.02601027488708496
Solve time:  0.00010442733764648438


Array([[1.50000000e+000, 1.71470296e+000, 1.96013749e+000,
        2.24070237e+000, 2.56142599e+000, 2.92805649e+000,
        3.34716475e+000, 3.82626220e+000, 4.37393541e+000,
        5.00000000e+000],
       [1.32893503e-022, 1.31967729e-023, 1.66217062e-024,
        2.56614940e-025, 4.61850699e-026, 9.01468594e-027,
        1.66878729e-027, 2.14571921e-028, 2.01230982e-029,
        8.12898390e-030],
       [1.09201369e-029, 7.83893501e-030, 6.02556175e-030,
        4.98268992e-030, 4.38492341e-030, 4.01854499e-030,
        3.76439572e-030, 3.59960874e-030, 3.61202493e-030,
        4.34857816e-191],
       [1.09201369e-029, 7.83893501e-030, 6.02556175e-030,
        4.98268992e-030, 4.38492341e-030, 4.01854499e-030,
        3.76439572e-030, 3.59960874e-030, 3.61202493e-030,
        4.34857816e-191]], dtype=float64)

In [7]:
mod.get_pdi_grids()

Array([[1.14776247e-004, 1.14776247e-003, 1.14776247e-002],
       [1.79507704e-180, 3.32704835e-029, 7.69015092e-032],
       [7.61656453e-183, 6.88650382e-031, 1.20633964e-034],
       [2.25062597e-184, 3.72505336e-032, 7.98129583e-036],
       [1.81990493e-182, 1.32219725e-030, 2.11540830e-034],
       [5.23902999e-184, 7.37756567e-032, 1.76685100e-035],
       [1.00000000e-200, 2.07791113e-037, 1.00000000e-200],
       [1.00000000e-200, 5.42095241e-038, 1.00000000e-200],
       [1.00000000e-200, 3.89011243e-100, 1.00000000e-200],
       [1.00000000e-200, 5.87007179e-144, 1.00000000e-200],
       [6.22968129e-182, 3.54311411e-030, 5.94929745e-034],
       [2.01147832e-192, 1.18925388e-035, 1.00000000e-200],
       [6.30950360e-181, 1.18740610e-029, 1.30081174e-032],
       [9.25721900e-184, 1.13249292e-031, 2.37792950e-035],
       [1.72201013e-186, 8.19466752e-034, 1.00000000e-200],
       [1.02890632e-180, 1.44151709e-029, 1.05423791e-031],
       [5.85469164e-183, 3.88415678e-031

In [10]:
rates_i = mod.pdi_rates(0.01)
print(rates_i)

{'dgnp': Array(2.33814052e-31, dtype=float64), 'tgnd': Array(2.03923383e-34, dtype=float64), 'tgnpn': Array(3.18770905e-35, dtype=float64), 'He3gpd': Array(3.3988849e-34, dtype=float64), 'He3gnpp': Array(4.80642814e-35, dtype=float64), 'He4gpt': Array(1.e-200, dtype=float64), 'He4gnHe3': Array(1.e-200, dtype=float64), 'He4gdd': Array(1.e-200, dtype=float64), 'He4gnpd': Array(1.e-200, dtype=float64), 'Li6gnpHe4': Array(9.72430938e-34, dtype=float64), 'Li6gX': Array(1.e-200, dtype=float64), 'Li7gtHe4': Array(3.23058209e-32, dtype=float64), 'Li7gnLi6': Array(6.10142597e-35, dtype=float64), 'Li7gnnpHe4': Array(2.81455349e-37, dtype=float64), 'Be7gHe3He4': Array(4.1189385e-31, dtype=float64), 'Be7gpLi6': Array(1.14787691e-34, dtype=float64), 'Be7gppnHe4': Array(1.40105552e-35, dtype=float64)}


In [14]:
pdi_rates = jnp.array([[1.79507704e-180, 7.61656453e-183, 2.25062597e-184,
        1.81990493e-182, 5.23902999e-184, 1.00000000e-200,
        1.00000000e-200, 1.00000000e-200, 1.00000000e-200,
        6.22968129e-182, 2.01147832e-192, 6.30950360e-181,
        9.25721900e-184, 1.72201013e-186, 1.02890632e-180,
        5.85469164e-183, 6.08196967e-185],
       [3.32704835e-029, 6.88650382e-031, 3.72505336e-032,
        1.32219725e-030, 7.37756567e-032, 2.07791113e-037,
        5.42095241e-038, 3.89011243e-100, 5.87007179e-144,
        3.54311411e-030, 1.18925388e-035, 1.18740610e-029,
        1.13249292e-031, 8.19466752e-034, 1.44151709e-029,
        3.88415678e-031, 1.30078527e-032],
       [7.69015092e-032, 1.20633964e-034, 7.98129583e-036,
        2.11540830e-034, 1.76685100e-035, 1.00000000e-200,
        1.00000000e-200, 1.00000000e-200, 1.00000000e-200,
        5.94929745e-034, 1.00000000e-200, 1.30081174e-032,
        2.37792950e-035, 1.00000000e-200, 1.05423791e-031,
        6.06419952e-035, 8.96912718e-037]], dtype=float)

In [ ]:
mod.get_cross_sections(E_grid)

In [7]:
T_range = mod.temperature_range()
print(T_range)
T_grid = jnp.linspace(T_range[0], T_range[1], 10)
T_grid

(Array(0.00011478, dtype=float64), Array(0.01147762, dtype=float64))


Array([0.00011478, 0.00137731, 0.00263985, 0.00390239, 0.00516493,
       0.00642747, 0.00769001, 0.00895255, 0.01021509, 0.01147762],      dtype=float64)

In [40]:
F_grid = InjSpec.get_spectrum(5, mod.get_source_0(), mod.get_source_cont(), 0.01)
print(F_grid)

[[1.50000000e+000 1.59812665e+000 1.70267252e+000 1.81405755e+000
  1.93272914e+000 2.05916396e+000 2.19386987e+000 2.33738793e+000
  2.49029462e+000 2.65320413e+000 2.82677081e+000 3.01169184e+000
  3.20870999e+000 3.41861663e+000 3.64225489e+000 3.88052306e+000
  4.13437820e+000 4.40483999e+000 4.69299477e+000 5.00000000e+000]
 [1.60455372e-022 5.17280504e-023 1.76108153e-023 6.33033932e-024
  2.39458428e-024 9.49226392e-025 3.92409523e-025 1.68256368e-025
  7.43635604e-026 3.36150448e-026 1.53893285e-026 7.03928272e-027
  3.15400008e-027 1.34271406e-027 5.17704114e-028 1.69364500e-028
  4.76871723e-029 1.87912465e-029 1.20112399e-029 8.12898390e-030]
 [1.82313982e-029 1.54346388e-029 1.32315604e-029 1.14935140e-029
  1.01202150e-029 9.03230913e-030 8.16631164e-030 7.47111755e-030
  6.90562269e-030 6.43716192e-030 6.04056109e-030 5.69762486e-030
  5.39686327e-030 5.13323761e-030 4.90781205e-030 4.72771318e-030
  4.60821597e-030 4.58287654e-030 4.75042215e-030 4.34857816e-191]
 [1.823

In [36]:
def F_s(log_E, i):
    E = jnp.exp(log_E)
    return jnp.exp(jnp.interp(jnp.log(E), jnp.log(F_grid[0]), jnp.log(F_grid[1]))) * E * mod.get_cross_section(E)[i]


In [38]:
F_s(jnp.log(5), 0)

Array(2.4251424e-34, dtype=float64)

In [25]:
a = 5
def cond_fun(state):
  # Define the condition for the loop to continue
  return state[0] < 10

def body_fun(state):
  # Define the operations to perform in each iteration
  return (state[0] + 1, state[1]**2)

init_val = 0  # Initial value for the loop variable

final_state = jax.lax.while_loop(cond_fun, body_fun, (0, 5))

print(final_state)  # Output: 10

(Array(10, dtype=int64, weak_type=True), Array(-766971542220705791, dtype=int64, weak_type=True))


In [26]:
S_0f, S_contf = mod.get_source_0(), mod.get_source_cont()

NE = jnp.array(jnp.log10(5/Emin)*150, int)
NE = jnp.maximum(NE, 150)
NE = 10

#number of species
N_X = 3

E_grid = jnp.logspace(jnp.log(Emin), jnp.log(5), NE, base=jnp.e)

# Generate the grid for the different species
X_grid = jnp.arange(N_X)

#rate (MeV)
R = jnp.array([[InjSpec.rate_x(X, E, 0.01) for E in E_grid] for X in X_grid])
#R = jnp.array([self.rate_x(X_grid, E, T) for E in E_grid])
ii, jj, kk, ll = jnp.meshgrid( X_grid, X_grid,E_grid, E_grid,indexing='ij')
k = jnp.stack([ll, kk, jj, ii], axis=-1)
k_true = jnp.array(k[:, :, :, :, 0] >= k[:, :, :, :, 1])
K = jnp.select([k_true], [jnp.array([[[[InjSpec.kernel_x(X, X_out, E, 0.01, Ep) for Ep in E_grid] for E in E_grid] for X_out in X_grid] for X in X_grid])])
S_0 = jnp.array([S0X(0.01) for S0X in S_0f])
S_cont = jnp.array([[SCX(E, 0.01) for E in E_grid] for SCX in S_contf])


In [28]:
from linx.injected_spec import set_spectra

In [42]:

N_X = len(R)
NE = len(E_grid)

dy = jnp.log(E_grid[-1]/Emin)/(NE-1)

#create the grid to store the spectrums
#F_grid = jnp.zeros((N_X, NE)) 

#calculate the last row, which is important for trapezoidal rule
FX_E0 = jnp.array([S_cont[X,-1]/R[X,-1] + jnp.sum(K[X,:,-1,-1]*S_0[:]/(R[:,-1]*R[X,-1])) for X in range(N_X)])

#F_grid = set_spectra(F_grid, -1, FX_E0)
#i  = (NE - 1) - 1 # start at the second to last index, NE-2

def cond_func(i_vec):
    return i_vec[0] >= 0

#@partial(jax.jit, static_argnums=(0,))
def body_func(i_vec):
#while i >= 0: # Counting down
    i = i_vec[0]
    B = jnp.zeros( (N_X, N_X) )
    a = jnp.zeros( (N_X,   ) )

    I = jnp.identity(N_X)
    # Calculate the matrix B and the vector a
    for X in jnp.arange(N_X):
        # Calculate B, : <--> Xp
        B = B.at[X,:].set(-.5*dy*E_grid[i]*K[X,:,i,i] + R[X,i]*I[X,:])

        # Calculate a
        a = a.at[X].set(S_cont[X,i])
        for Xp in jnp.arange(N_X):
            a = a.at[X].add(K[X,Xp,i,-1]*S_0[Xp]/R[Xp,-1] + .5*dy*E_grid[-1]*K[X,Xp,i,-1]*i_vec[1][Xp,-1])
            for j in range(i, NE-1): # Goes from i+1 to NE-2
                a = a.at[X].add(dy*E_grid[j]*K[X,Xp,i,j]*i_vec[1][Xp,j])

    # Solve the system of linear equations of the form BF = a
    F_grid = set_spectra(i_vec[1], i,jnp.linalg.solve(B, a))

    return (i-1, F_grid)

F_grid = jax.lax.while_loop(cond_func, body_func, ((NE - 1) - 1, set_spectra(jnp.zeros((N_X, NE)), -1, FX_E0)))[1]

approx_zero = 1e-200
# Remove potential zeros
F_grid = F_grid.reshape( N_X*NE )
F_grid = F_grid.at[:].set(jnp.where(F_grid > approx_zero, F_grid, approx_zero))
#for i, f in enumerate(F_grid):
    #if f < approx_zero:
    #    F_grid[i] = approx_zero
#    F_grid.at[i].set(f < approx_zero, f, approx_zero)
F_grid = F_grid.reshape( (N_X, NE) )

# Define the output array...
sol = jnp.zeros( (N_X+1, NE) )
# ...and fill it
sol = sol.at[0     , :].set(E_grid)
sol = sol.at[1:N_X+1, :].set(F_grid)

sol


NameError: name 'R' is not defined

In [107]:
body_func((NE - 1, set_spectra(jnp.zeros((N_X, NE)), -1, FX_E0)))

ValueError: Non-hashable static arguments are not supported. An error occurred while trying to hash an object of type <class 'tuple'>, (9, Array([[0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
        0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
        0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
        8.12898311e-030],
       [0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
        0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
        0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
        4.34857814e-191],
       [0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
        0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
        0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
        4.34857814e-191]], dtype=float64)). The error was:
TypeError: unhashable type: 'jaxlib.xla_extension.ArrayImpl'


In [161]:
jnp.arange(1, 3)

Array([1, 2], dtype=int64)

In [158]:
#@partial(jax.jit, static_argnums=0)
@jax.jit
def test_func(i):
    l = i[0] 
    for j in jnp.arange(l, 3):
        t = E_grid[j]
    return t


In [160]:
test_func((0, E_grid))

ConcretizationTypeError: Abstract tracer value encountered where concrete value is expected: traced array with shape int64[]
It arose in the jnp.arange argument 'start'
The error occurred while tracing the function test_func at /tmp/ipykernel_376561/3703535805.py:2 for jit. This concrete value was not available in Python because it depends on the value of the argument i[0].

See https://jax.readthedocs.io/en/latest/errors.html#jax.errors.ConcretizationTypeError

In [55]:
%timeit InjSpec.inverse_compton_kernel_photon(5, 0.01, 8)

313 μs ± 7.46 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [54]:
%timeit InjSpec.inverse_compton_kernel_photon(5, 0.01, 8)

340 μs ± 21.6 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [23]:
print("Photon Rates Test")
print("---------------")
print("double photon", InjSpec.dphoton_pair_prod_rate(5, 0.01))
print("photonphoton", InjSpec.photon_photon_scattering_rate(5, 0.01))
print("BH", InjSpec.bethe_heitler_pair_prod_rate(1, 0.01), InjSpec.bethe_heitler_pair_prod_rate(3, 0.01), InjSpec.bethe_heitler_pair_prod_rate(5, 0.01))
print("CS", InjSpec.compton_scattering_rate(5, 0.01))
print("total", InjSpec.total_rate_photon(5, 0.01))
print()
print("Photon Kernel Test")
print("---------------")
print("photonphoton", InjSpec.photon_photon_scattering_kernel(5, 0.01, 8))
print("CS", InjSpec.compton_scattering_kernel_photon(5, 0.01, 8))
print("Inverse CS", InjSpec.inverse_compton_kernel_photon(5, 0.01, 8))
print("total", InjSpec.total_kernel_photon(5, 0.01, 8, 0), InjSpec.total_kernel_photon(5, 0.01, 8, 1), InjSpec.total_kernel_photon(5, 0.01, 8, 2))

Photon Rates Test
---------------
double photon 1.7273805573001668e-12
photonphoton 9.528579442514193e-18
BH 0.0 1.93931715860427e-22 4.226659078975712e-22
CS 2.7877735519401764e-20
total 1.7273901141800107e-12

Photon Kernel Test
---------------
photonphoton 7.283939142849163e-18
CS 1.4405883400723104e-21
Inverse CS 1.6870306215985535e-11
total 7.285379731189236e-18 1.6870306215985535e-11 1.6870306215985535e-11


In [16]:
print("Electron Rates Test")
print("---------------")
print("Inverse CS", InjSpec.inverse_compton_rate(5, 0.01))
print("total", InjSpec.total_lepton_rate(5, 0.01))
print()
print("Electron Kernels Test")
print("---------------")
print("Double photon", InjSpec.dphoton_pair_prod_kernel(5, 0.01, 8))
print("BH", InjSpec.bethe_heitler_pair_prod_kernel(5, 0.01, 8))
print("CS", InjSpec.compton_scattering_kernel_electron(5, 0.01, 8))
print("Inverse CS", InjSpec.inverse_compton_kernel_lepton(5, 0.01, 8))
print("total", InjSpec.total_kernel_electron(5, 0.01, 8, 0), InjSpec.total_kernel_electron(5, 0.01, 8, 1), InjSpec.total_kernel_electron(5, 0.01, 8, 2))

Electron Rates Test
---------------
Inverse CS 2.2996022345164137e-10
total 2.2996022345164137e-10

Electron Kernels Test
---------------
Double photon 2.3844163031161436e-12
BH 1.1385678331397403e-22
CS 1.7159611430076526e-21
Inverse CS 2.8852488761155606e-11
total 2.3844163049459616e-12 2.8852488761155606e-11 0.0


In [140]:
print("Positron Rates Test")
print("---------------")
print("Inverse CS", InjSpec.inverse_compton_rate(5, 0.01))
print("total", InjSpec.total_lepton_rate(5, 0.01))
print()
print("Positron Kernels Test")
print("---------------")
print("Double photon", InjSpec.dphoton_pair_prod_kernel(5, 0.01, 8))
print("BH", InjSpec.bethe_heitler_pair_prod_kernel(5, 0.01, 8))
print("CS", InjSpec.compton_scattering_kernel_electron(5, 0.01, 8))
print("Inverse CS", InjSpec.inverse_compton_kernel_lepton(5, 0.01, 8))
print("total", InjSpec.total_kernel_positron(5, 0.01, 8, 0), InjSpec.total_kernel_positron(5, 0.01, 8, 1), InjSpec.total_kernel_positron(5, 0.01, 8, 2))

Positron Rates Test
---------------
Inverse CS 2.299602280267959e-10
total 2.299602280267959e-10

Positron Kernels Test
---------------
Double photon 2.384416303116144e-12
BH 1.1385678331397403e-22
CS 1.7159611430076522e-21
Inverse CS 2.885248876115561e-11
total 2.384416304945962e-12 0 2.885248876115561e-11


In [124]:
print("Photon Rates Test NEW")
print("---------------")
print("double photon")
%timeit InjSpec.dphoton_pair_prod_rate(5, 0.01)
print("photonphoton")
%timeit InjSpec.photon_photon_scattering_rate(5, 0.01)
print("BH")
%timeit InjSpec.bethe_heitler_pair_prod_rate(1, 0.01)
print("CS")
%timeit InjSpec.compton_scattering_rate(5, 0.01)
print("total")
%timeit InjSpec.total_rate_photon(5, 0.01)
print()
print("Photon Kernel Test")
print("---------------")
print("photonphoton")
%timeit InjSpec.photon_photon_scattering_kernel(5, 0.01, 8)
print("CS")
%timeit InjSpec.compton_scattering_kernel_photon(5, 0.01, 8)
print("Inverse CS")
%timeit InjSpec.inverse_compton_kernel_photon(5, 0.01, 8)
print("total")
%timeit InjSpec.total_kernel_photon(5, 0.01, 8, 0)

Photon Rates Test NEW
---------------
double photon
19.3 ms ± 382 μs per loop (mean ± std. dev. of 7 runs, 1 loop each)
photonphoton
234 μs ± 3.7 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
BH
234 μs ± 1.26 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
CS
233 μs ± 1.21 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
total
20.2 ms ± 1.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

Photon Kernel Test
---------------
photonphoton
236 μs ± 1.46 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
CS
237 μs ± 976 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
Inverse CS
378 μs ± 127 μs per loop (mean ± std. dev. of 7 runs, 1 loop each)
total
242 μs ± 1.71 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [142]:
print("Photon Kernel Test")
print("---------------")
print("double photon")
%timeit InjSpec.dphoton_pair_prod_rate(5, 0.01)
print("photonphoton")
%timeit InjSpec.photon_photon_scattering_rate(5, 0.01)
print("BH")
%timeit InjSpec.bethe_heitler_pair_prod_rate(1, 0.01)
print("CS")
%timeit InjSpec.compton_scattering_rate(5, 0.01)
print("total")
%timeit InjSpec.total_rate_photon(5, 0.01)
print()
print("Photon Kernel Test")
print("---------------")
print("photonphoton")
%timeit InjSpec.photon_photon_scattering_kernel(5, 0.01, 8)
print("CS")
%timeit InjSpec.compton_scattering_kernel_photon(5, 0.01, 8)
print("Inverse CS")
%timeit InjSpec.inverse_compton_kernel_photon(5, 0.01, 8)
print("total")
%timeit InjSpec.total_kernel_photon(5, 0.01, 8, 0)

Photon Kernel Test
---------------
double photon
130 ms ± 45.2 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
photonphoton
87.7 μs ± 1.39 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
BH
50.2 μs ± 1.37 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
CS
161 μs ± 1.81 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
total
88.4 ms ± 3.31 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)

Photon Kernel Test
---------------
photonphoton
101 μs ± 5.2 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
CS
119 μs ± 5.3 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
Inverse CS
1.28 ms ± 33.1 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
total
274 μs ± 3.49 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [43]:
print("Electron Rates Test NEW")
print("---------------")
print("Inverse CS")
%timeit InjSpec.inverse_compton_rate(5, 0.01)
print("total")
%timeit InjSpec.total_lepton_rate(5, 0.01)
print()
print("Electron Kernels Test")
print("---------------")
print("Double photon")
%timeit InjSpec.dphoton_pair_prod_kernel(5, 0.01, 8)
print("BH")
%timeit InjSpec.bethe_heitler_pair_prod_kernel(5, 0.01, 8)
print("CS")
%timeit InjSpec.compton_scattering_kernel_electron(5, 0.01, 8)
print("Inverse CS")
%timeit InjSpec.inverse_compton_kernel_lepton(5, 0.01, 8)
print("total")
%timeit InjSpec.total_kernel_electron(5, 0.01, 8, 0)

Electron Rates Test NEW
---------------
Inverse CS
23.8 ms ± 4.26 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
total
17.7 ms ± 392 μs per loop (mean ± std. dev. of 7 runs, 1 loop each)

Electron Kernels Test
---------------
Double photon
386 μs ± 122 μs per loop (mean ± std. dev. of 7 runs, 1 loop each)
BH
285 μs ± 7.11 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
CS
270 μs ± 9.01 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
Inverse CS
368 μs ± 132 μs per loop (mean ± std. dev. of 7 runs, 1 loop each)
total
1.98 ms ± 74.7 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [143]:
print("Electron Rates Test")
print("---------------")
print("Inverse CS")
%timeit InjSpec.inverse_compton_rate(5, 0.01)
print("total")
%timeit InjSpec.total_lepton_rate(5, 0.01)
print()
print("Electron Kernels Test")
print("---------------")
print("Double photon")
%timeit InjSpec.dphoton_pair_prod_kernel(5, 0.01, 8)
print("BH")
%timeit InjSpec.bethe_heitler_pair_prod_kernel(5, 0.01, 8)
print("CS")
%timeit InjSpec.compton_scattering_kernel_electron(5, 0.01, 8)
print("Inverse CS")
%timeit InjSpec.inverse_compton_kernel_lepton(5, 0.01, 8)
print("total")
%timeit InjSpec.total_kernel_electron(5, 0.01, 8, 0)

Electron Rates Test
---------------
Inverse CS
291 ms ± 6.93 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
total
294 ms ± 6.57 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

Electron Kernels Test
---------------
Double photon
1.43 ms ± 69.3 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
BH
1.47 ms ± 33.3 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
CS
172 μs ± 1.42 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
Inverse CS
1.3 ms ± 5.3 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
total
3.25 ms ± 128 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [144]:
print("Positron Rates Test")
print("---------------")
print("Inverse CS")
%timeit InjSpec.inverse_compton_rate(5, 0.01)
print("total")
%timeit InjSpec.total_lepton_rate(5, 0.01)
print()
print("Positron Kernels Test")
print("---------------")
print("Double photon")
%timeit InjSpec.dphoton_pair_prod_kernel(5, 0.01, 8)
print("BH")
%timeit InjSpec.bethe_heitler_pair_prod_kernel(5, 0.01, 8)
print("CS")
%timeit InjSpec.compton_scattering_kernel_electron(5, 0.01, 8)
print("Inverse CS")
%timeit InjSpec.inverse_compton_kernel_lepton(5, 0.01, 8)
print("total")
%timeit InjSpec.total_kernel_positron(5, 0.01, 8, 0)

Positron Rates Test
---------------
Inverse CS
285 ms ± 1.68 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
total
283 ms ± 1.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

Positron Kernels Test
---------------
Double photon
1.45 ms ± 8.57 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
BH
1.5 ms ± 9.4 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
CS
180 μs ± 2.85 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
Inverse CS
1.38 ms ± 49.6 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
total
3.45 ms ± 142 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [100]:
%timeit InjSpec.dphoton_pair_prod_rate(5, 0.01)

102 ms ± 34.3 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [101]:
InjSpec.dphoton_pair_prod_rate(5, 0.01)

1.7273815531413682e-12

In [25]:
a = jnp.linspace(1, 10, 10)
a = jnp.where(a > 5, 0, a)
print(a)

[1. 2. 3. 4. 5. 0. 0. 0. 0. 0.]


In [ ]:
from scipy.integrate import dblquad, quad
from scipy.special import erf, jv


def h(t, z):
    return f(t) * g(z)


def f(t):
    return 0.5 * t * (erf(t - a) - 1) * jv(0, q * t)


def g(z):
    return math.exp(-((z - a) ** 2)/(2 * (s ** 2)))


def h1(z):
    return integral_of_f(z) * g(z)

def integral_of_f(z):
    return quad(f, 0, 2 * z)[0]  # here abserr is discarded


if __name__ == '__main__':
    a, q, s = 0, 2, 3  # set the constants

    result, abserr = dblquad(h, 0, 60, lambda z: 0, lambda z: 2 * z)
    print(f'result: {result}, abserr: {abserr}')

    result1 = quad(h1, 0, 60)[0]  # abserr here for result1 is not valid since abserr is already discarded in the function integral_of_f
    print(f'result1: {result1}')


result: -0.21705286423629183, abserr: 1.4361242750875712e-08
result1: -0.21705286423678177
result2: 0.013105371081178754


In [11]:
@jax.jit
def f_thermal_photon_spec(ep, T):
    return (ep**2)/jnp.pi**2 * 1/(jnp.exp(ep/T) - 1)
#
@jax.jit
def dp_integrand(s, ep, T):
    s = jnp.exp(s)
    ep = jnp.exp(ep)
    b = jnp.sqrt(1 - (4*me**2)/s)
    dp_cross_section = jnp.pi * aFS**2 /(2*me**2) * (1-b**2) * ((3-b**4) * jnp.log((1+b)/(1-b)) - 2*b *(2-b**2))  
    return f_thermal_photon_spec(ep, T)/ep**2 * dp_cross_section * s *ep *s  

@jax.jit
def dphoton_pair_prod_rate(E, T):

    #Check if incoming energy is greater than me^2/(22T)
    #acropolis uses 50T for a smaller threshold
    """    if (type(E) != float) and (type(E) != int):
        E = jnp.where(E < (me**2)/(50*T), 0, E)
    else:
    """
    #if E < (me**2)/(50*T):
    #    return 0
    
    @jax.jit
    def inside_int(s):
        s = jnp.exp(s)
        b = jnp.sqrt(1 - (4*me**2)/s)
        dp_cross_section = jnp.pi * aFS**2 /(2*me**2) * (1-b**2) * ((3-b**4) * jnp.log((1+b)/(1-b)) - 2*b *(2-b**2))  
        return dp_cross_section * s *s  
    
    @jax.jit
    def integral_of_inside(ep):
        return quadgk(inside_int, [jnp.log(4*me**2), jnp.log(4*E *ep)], epsrel = eps/10, epsabs=0)[0]
    
    @jax.jit
    def outside_integral(ep, T):
        ep = jnp.exp(ep)
        return integral_of_inside(ep) * f_thermal_photon_spec(ep, T)/ep**2 *ep



    ep_ll = jnp.log((me**2)/E)
    ep_ul = jnp.log(Ephb_T_max*T)

    #I_dp_pp = dblquad(dp_integrand, ep_ll, ep_ul, lambda x: jnp.log(4*me**2), lambda x: jnp.log(4*E) + x, epsrel = eps, epsabs=0, args=(T,)) 
    I_dp_pp = quadgk(outside_integral, [ep_ll, ep_ul], epsrel = eps, epsabs=0, args=(T,))
    return 1/(8*E**2) * I_dp_pp[0]

In [97]:
@jax.jit
def icre_integrand(E_ph, ep, E, T):
    #equation B.23 in Hufnagel 2018
    F = F_func(E_ph, E, ep)
    return f_thermal_photon_spec(ep, T)/ep * F
@jax.jit
def F_func(E, Ep, ep):   
        """Function used for calculating rates and kernels related to inverse compton scattering 

        Parameters
        ----------
        E: float 
            Energy of outgoing particle (MeV)
        Ep: float 
            E', energy of incoming particle (MeV)
        ep: float
            epsilon, energy of thermal bath

        Returns
        -------
        float 
            F(E, E', epsilon), which is (dimensionless)
        """
        
        
        #should never happen if limits are right
        '''
        if not (ep <= E <= (4*ep*Ep**2)/(me**2 + 4*ep*Ep)):
            return 0
        '''
        
        G_ep = 4*ep*Ep/(me**2)
        q = E/(G_ep*(Ep-E))
        F = 2*q*jnp.log(q) + (1+2*q)*(1-q) + G_ep**2 * q**2 * (1 - q)/(2+2*G_ep*q)

        return F

@jax.jit
def inverse_compton_rate(E, T):
        """ 
        Inverse Compton Scattering Rate Eq. B.24 in Hufnagel 2018
        
        Parameters
        ----------
        E: float
            Energy of outgoing particle (MeV)
        T: float
            Temperature of the thermal bath (MeV)

        Returns
        -------
        float
            Gamma^IC_e+- (MeV)
            
        """
        @jax.jit
        def inside_int(E_ph, E, ep):
            return F_func(E_ph, E, ep)
        
        @jax.jit
        def integral_of_inside(ep, E):
            return quadgk(inside_int, [ep, 4*ep*E**2 /(me**2 + 4*ep*E)], epsrel = eps/10, epsabs=0, args = (E, ep))[0]

        @jax.jit
        def outside_integral(ep, T, E):
            return integral_of_inside(ep, E) * f_thermal_photon_spec(ep, T)/ep
        
        ep_ll = 0
        ep_ul = jnp.minimum(E - (me**2)/(4*E), Ephb_T_max*T)
        #E_ph_ll = lambda x: x
        #E_ph_ul = lambda x: 4*x*E**2 /(me**2 + 4*x*E)

        #return 2*jnp.pi*aFS**2 /E**2 * dblquad(icre_integrand, ep_ll, ep_ul, E_ph_ll, E_ph_ul,  epsrel=eps, epsabs=0, args=(E, T))[0]
        return 2*jnp.pi*aFS**2 /E**2 * quadgk(outside_integral, [ep_ll, ep_ul], epsrel=eps, epsabs=0, args=(T, E))[0]

#@jax.jit
def inverse_compton_rate_vmap(E, T):
        """ 
        Inverse Compton Scattering Rate Eq. B.24 in Hufnagel 2018
        
        Parameters
        ----------
        E: float
            Energy of outgoing particle (MeV)
        T: float
            Temperature of the thermal bath (MeV)

        Returns
        -------
        float
            Gamma^IC_e+- (MeV)
            
        """
        @jax.jit
        def inside_int(E_ph, E, ep):
            return F_func(E_ph, E, ep)
        
        @jax.jit
        def integral_of_inside(ep, E):
            I_i =  jax.vmap(quadgk, in_axes=(None, 0, 1, None, None, None))(inside_int, [ep, 4*ep*E**2 /(me**2 + 4*ep*E)], jnp.array((E, ep)), False, 0,  eps)
            return I_i[0]
        @jax.jit
        def outside_integral(ep, T, E):
            return integral_of_inside(ep, E) * f_thermal_photon_spec(ep, T)/ep
        
        
        ep_ul = jnp.minimum(E - (me**2)/(4*E), Ephb_T_max*T)
        ep_ll = jnp.zeros_like(ep_ul)
        #E_ph_ll = lambda x: x
        #E_ph_ul = lambda x: 4*x*E**2 /(me**2 + 4*x*E)

        #return 2*jnp.pi*aFS**2 /E**2 * dblquad(icre_integrand, ep_ll, ep_ul, E_ph_ll, E_ph_ul,  epsrel=eps, epsabs=0, args=(E, T))[0]
        I =  2*jnp.pi*aFS**2 /E**2 * jax.vmap(quadgk, in_axes=(None, 0, 0, None, None, None))(outside_integral, [ep_ll, ep_ul], jnp.array((T, E)), False, 0, eps)
        return I[0]

In [98]:
def printing(a, b, c, d, e):
    return (a, b, c, d, e)

In [99]:
jax.vmap(printing, in_axes=[0, 1, None, None, 0])(jnp.arange(5), jnp.zeros((2, 5)), 0, 0, jnp.arange(5))

(Array([0, 1, 2, 3, 4], dtype=int64),
 Array([[0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.]], dtype=float64),
 Array([0, 0, 0, 0, 0], dtype=int64, weak_type=True),
 Array([0, 0, 0, 0, 0], dtype=int64, weak_type=True),
 Array([0, 1, 2, 3, 4], dtype=int64))

In [100]:
import time
test_energies = jnp.linspace(1, 10, 10)
inverse_compton_rate_vmap(test_energies, jnp.full_like(test_energies, 0.01))
start = time.time()
print("hello")
end = time.time()
print(end - start)

ValueError: vmap got inconsistent sizes for array axes to be mapped:
  * most axes (2 of them) had size 10, e.g. axis 0 of argument interval[0] of type float64[10];
  * one axis had size 2: axis 0 of argument args of type float64[2,10]

In [9]:
@jax.jit
def loopfunc(E_list):
    for E_i in E_list:
        print(inverse_compton_rate(E_i, 0.01))

In [27]:
start = time.time()
loopfunc(test_energies)
end = time.time()
print("time", end - start)

Traced<ShapedArray(float64[])>with<DynamicJaxprTrace(level=1/0)>
Traced<ShapedArray(float64[])>with<DynamicJaxprTrace(level=1/0)>
Traced<ShapedArray(float64[])>with<DynamicJaxprTrace(level=1/0)>
Traced<ShapedArray(float64[])>with<DynamicJaxprTrace(level=1/0)>
Traced<ShapedArray(float64[])>with<DynamicJaxprTrace(level=1/0)>
Traced<ShapedArray(float64[])>with<DynamicJaxprTrace(level=1/0)>
Traced<ShapedArray(float64[])>with<DynamicJaxprTrace(level=1/0)>
Traced<ShapedArray(float64[])>with<DynamicJaxprTrace(level=1/0)>
Traced<ShapedArray(float64[])>with<DynamicJaxprTrace(level=1/0)>
Traced<ShapedArray(float64[])>with<DynamicJaxprTrace(level=1/0)>
time 0.029423952102661133


In [16]:
X_grid = jnp.arange(3)
E_grid = jnp.logspace(jnp.log(Emin), jnp.log(5), 50, base=jnp.e)
x1, x2 = jnp.meshgrid(X_grid, E_grid, indexing='ij')



In [11]:
x1.flatten()

Array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2], dtype=int64)

In [12]:
x2.flatten()

Array([1.5       , 1.71470296, 1.96013749, 2.24070237, 2.56142599,
       2.92805649, 3.34716475, 3.8262622 , 4.37393541, 5.        ,
       1.5       , 1.71470296, 1.96013749, 2.24070237, 2.56142599,
       2.92805649, 3.34716475, 3.8262622 , 4.37393541, 5.        ,
       1.5       , 1.71470296, 1.96013749, 2.24070237, 2.56142599,
       2.92805649, 3.34716475, 3.8262622 , 4.37393541, 5.        ],      dtype=float64)

In [21]:
jax.vmap(InjSpec.rate_x, in_axes=(0, 0, None))(x1.flatten(), x2.flatten(), 0.01) 

Array([1.48929702e-17, 2.23273086e-17, 3.32049408e-17, 4.89639844e-17,
       7.15743113e-17, 1.03707191e-16, 1.48949634e-16, 2.12072299e-16,
       2.99360829e-16, 4.19021320e-16, 5.81671436e-16, 8.00929140e-16,
       1.09411134e-15, 1.48305469e-15, 1.99506992e-15, 2.66404121e-15,
       3.53167897e-15, 4.64893389e-15, 6.07757683e-15, 7.89194558e-15,
       1.01808586e-14, 1.30496843e-14, 1.66225606e-14, 2.10447458e-14,
       2.64850787e-14, 3.31385238e-14, 4.12287682e-14, 5.10108328e-14,
       6.27736583e-14, 7.68426184e-14, 9.35819122e-14, 1.13396783e-13,
       1.36735514e-13, 1.64091140e-13, 1.96002829e-13, 2.33056883e-13,
       2.75887300e-13, 3.25175868e-13, 3.81651740e-13, 4.46090467e-13,
       5.19312457e-13, 6.02180854e-13, 6.95598813e-13, 8.00506195e-13,
       9.17875668e-13, 1.04870826e-12, 1.19402836e-12, 1.35487824e-12,
       1.53231215e-12, 1.72738995e-12, 2.84328170e-10, 2.84244196e-10,
       2.84089872e-10, 2.83867483e-10, 2.83579263e-10, 2.83227405e-10,
      

In [22]:
jnp.array([[InjSpec.rate_x(X, E, 0.01) for E in E_grid] for X in X_grid])

Array([[1.48929702e-17, 2.23273086e-17, 3.32049408e-17, 4.89639844e-17,
        7.15743113e-17, 1.03707191e-16, 1.48949634e-16, 2.12072299e-16,
        2.99360829e-16, 4.19021320e-16, 5.81671436e-16, 8.00929140e-16,
        1.09411134e-15, 1.48305469e-15, 1.99506992e-15, 2.66404121e-15,
        3.53167897e-15, 4.64893389e-15, 6.07757683e-15, 7.89194558e-15,
        1.01808586e-14, 1.30496843e-14, 1.66225606e-14, 2.10447458e-14,
        2.64850787e-14, 3.31385238e-14, 4.12287682e-14, 5.10108328e-14,
        6.27736583e-14, 7.68426184e-14, 9.35819122e-14, 1.13396783e-13,
        1.36735514e-13, 1.64091140e-13, 1.96002829e-13, 2.33056883e-13,
        2.75887300e-13, 3.25175868e-13, 3.81651740e-13, 4.46090467e-13,
        5.19312457e-13, 6.02180854e-13, 6.95598813e-13, 8.00506195e-13,
        9.17875668e-13, 1.04870826e-12, 1.19402836e-12, 1.35487824e-12,
        1.53231215e-12, 1.72738995e-12],
       [2.84328170e-10, 2.84244196e-10, 2.84089872e-10, 2.83867483e-10,
        2.83579263e-10,

In [118]:
ii, jj, kk, ll = jnp.meshgrid( X_grid, X_grid,E_grid, E_grid,indexing='ij')

In [160]:
a =  jax.vmap(InjSpec.kernel_x, in_axes=(0, 0, 0, None, 0))(ii.flatten(), jj.flatten(), kk.flatten(), 0.01, ll.flatten())
a = jnp.reshape(a, (3, 3, 10, 10))
a = jnp.nan_to_num(a)


In [161]:
b = jnp.array([[[[InjSpec.kernel_x(X, X_out, E, 0.01, Ep) for Ep in E_grid] for E in E_grid] for X_out in X_grid] for X in X_grid])
b = jnp.nan_to_num(b)

In [ ]:
%timeit jax.vmap(InjSpec.rate_x, in_axes=(0, 0, None))(x1.flatten(), x2.flatten(), 0.01) 

In [35]:
inverse_compton_rate(test_energies, 0.01)

ValueError: All input arrays must have the same shape.

In [39]:
start = time.time()
print(jax.vmap(inverse_compton_rate, in_axes = (0, None))(test_energies, 0.01))
end = time.time()
print("time", end - start)

[2.73355334e-10 2.79377492e-10 2.61624118e-10 2.44565398e-10
 2.29960223e-10 2.17541271e-10 2.06875252e-10 1.97600359e-10
 1.89440259e-10 1.82186297e-10]
time 2.150092363357544


In [37]:
%timeit loopfunc(test_energies)

2.44 μs ± 71.4 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [36]:
%timeit print(jax.vmap(inverse_compton_rate, in_axes = (0, None))(test_energies, 0.01))

[2.73355334e-10 2.79377492e-10 2.61624118e-10 2.44565398e-10
 2.29960223e-10 2.17541271e-10 2.06875252e-10 1.97600359e-10
 1.89440259e-10 1.82186297e-10]
[2.73355334e-10 2.79377492e-10 2.61624118e-10 2.44565398e-10
 2.29960223e-10 2.17541271e-10 2.06875252e-10 1.97600359e-10
 1.89440259e-10 1.82186297e-10]
[2.73355334e-10 2.79377492e-10 2.61624118e-10 2.44565398e-10
 2.29960223e-10 2.17541271e-10 2.06875252e-10 1.97600359e-10
 1.89440259e-10 1.82186297e-10]
[2.73355334e-10 2.79377492e-10 2.61624118e-10 2.44565398e-10
 2.29960223e-10 2.17541271e-10 2.06875252e-10 1.97600359e-10
 1.89440259e-10 1.82186297e-10]
[2.73355334e-10 2.79377492e-10 2.61624118e-10 2.44565398e-10
 2.29960223e-10 2.17541271e-10 2.06875252e-10 1.97600359e-10
 1.89440259e-10 1.82186297e-10]
[2.73355334e-10 2.79377492e-10 2.61624118e-10 2.44565398e-10
 2.29960223e-10 2.17541271e-10 2.06875252e-10 1.97600359e-10
 1.89440259e-10 1.82186297e-10]
[2.73355334e-10 2.79377492e-10 2.61624118e-10 2.44565398e-10
 2.29960223e-10

In [22]:
dphoton_pair_prod_rate(5, 0.01)

Array(1.72738056e-12, dtype=float64)

In [110]:
%timeit dphoton_pair_prod_rate(5, 0.01)

21.2 ms ± 513 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [35]:
%timeit dphoton_pair_prod_rate(5, 0.01)

144 ms ± 39 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [24]:
%timeit dphoton_pair_prod_rate(5, 0.01)

238 ms ± 4.82 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [78]:
def test_func(x, y, z):
    return x*y

dblquad(test_func, 0, 1, lambda x: 0, lambda x: 4.*x , args=(2, ))

(2.0, 8.824051529244462e-14)

In [85]:
from quadax import quadgk, quadcc, quadts, romberg

In [11]:
gaussian = lambda x: 1/jnp.sqrt(jnp.pi) * jnp.exp(-x**2)

In [14]:
quadgk(gaussian, [0, 1])

(Array(0.4213504, dtype=float64),
 QuadratureInfo(err=Array(4.67792912e-15, dtype=float64), neval=Array(21, dtype=int64, weak_type=True), status=Array(0, dtype=int64, weak_type=True), info=None))

In [50]:
%timeit quadgk(gaussian, [0, 1])

335 ms ± 34.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [59]:
%timeit quadcc(gaussian, [0, 1])

408 ms ± 39.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [60]:
%timeit quadts(gaussian, [0, 1])

394 ms ± 33.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [51]:
%timeit quad(gaussian, 0, 1)

801 μs ± 26.6 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [62]:
%timeit romberg(gaussian, [0, 1])

206 ms ± 18.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [56]:
%timeit jc.scipy.integrate.romb(gaussian, 0, 1)

179 ms ± 2.49 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [13]:
inverse_compton_rate(3, 0.01)

Array(2.61624118e-10, dtype=float64)

In [120]:
inverse_compton_rate(3, 0.01)

Array(2.61624118e-10, dtype=float64)

In [119]:
inverse_compton_rate(3, 0.01)

Array(2.61624118e-10, dtype=float64)

In [35]:
def photon_photon_scattering_rate(E, T):

    exp_fac = jnp.exp(-E*T/me**2)

    return 15568*(jnp.pi**3)/(3189375) * aFS**4 * me * (E/me)**3 * (T/me)**6 * exp_fac


In [26]:
import math

In [27]:
def _rate_photon_photon( E, T):
    #if E > me2/T:
    #    return 0.
    expf = math.exp( -E*T/me**2 )

    return 0.151348 * (aFS**4.) * me * (E/me)**3. * (T/me)**6. * expf

@jax.jit
def photon_photon_scattering_rate_jit(E, T):

    exp_fac = jnp.exp(-E*T/me**2)

    return 15568*(jnp.pi**3)/(3189375) * aFS**4 * me * (E/me)**3 * (T/me)**6 * exp_fac

@jax.jit
def photon_photon_scattering_rate_lax(E, T):

    exp_fac = jax.lax.exp(-E*T/me**2)

    return 15568*(jnp.pi**3)/(3189375) * aFS**4 * me * (E/me)**3 * (T/me)**6 * exp_fac


In [28]:
start = time.time()
#for i in range(10000):
photon_photon_scattering_rate_lax(jnp.full((100), 5), jnp.full((100), 0.01))
end = time.time()
print(end-start)

0.021364212036132812


In [29]:
start = time.time()
#for i in range(10000):
photon_photon_scattering_rate_jit(jnp.full((100), 5), jnp.full((100), 0.01))
end = time.time()
print(end-start)

0.020008325576782227


In [47]:
n = 1000
start = time.time()
#for i in range(10000):
jax.vmap(photon_photon_scattering_rate_jit, in_axes=(0, 0))(jnp.full((n), 5), jnp.full((n), 0.01))
end = time.time()
print(end-start)

0.001374959945678711


In [48]:
start = time.time()
for i in range(n):
    _rate_photon_photon(5, 0.01)
end = time.time()
print(end-start)

0.0003592967987060547
